In [2]:
!pip install python-dotenv
!git init
!touch .gitignore
!echo "info.env" >> .gitignore
!git check-ignore -v .env


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [76]:
import pandas as pd
import http.client
import json
import requests
import os
from dotenv import load_dotenv
import re

# Load environment variables
load_dotenv("info.env")

# Access credentials
authUsername = os.getenv("API_USERNAME")
authPassword = os.getenv("API_PASSWORD")

### Pre-filtering based on inventory ###
# Search based on specific filters
page = 5
page_size = 50
craft = 'crochet'
knit_gauge = 5
weight = 'DK'
query = 'dragon'

# Define URL for the API request
url = 'https://api.ravelry.com/patterns/search.json?page={}&page_size={}&craft={}'.format(page, page_size, craft)
# Make request
r = requests.get(url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
# Close connection
r.close()

def getPatterns():
  yarndf = pd.DataFrame()
  keydf = pd.DataFrame()
  if r.status_code == 200:
      data = r.json()
      # Extract pattern ID(s) from the search results
      if 'patterns' in data and len(data['patterns']) > 0:
        for i in range(len(data['patterns'])):
            pattern_id = data['patterns'][i]['id']

            # Define URL to get pattern details
            pattern_url = f'https://api.ravelry.com/patterns/{pattern_id}.json'

            # Make the request for pattern details
            pattern_response = requests.get(pattern_url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
            pattern_response.close()

            # If patterns found, collect the necessary yarn info
            if pattern_response.status_code == 200:
                pattern_data = pattern_response.json()
                try:
                  #print(json.dumps(json.loads(pattern_response.text), indent=4))
                  yarnData = pattern_data['pattern']['yarn_weight']
                  categories = pattern_data['pattern']['pattern_categories']

                  # Extract yardage for minimum/maximums
                  yardage = ['0', '0']
                  extracted = re.findall(r'\d+', pattern_data['pattern']['yardage_description'])

                  if len(extracted) == 2:
                    yardage = extracted
                  elif len(extracted) == 1:
                    yardage[0] = extracted[0]
                    yardage[1] = extracted[0]

                  # Get project name and add all to dataframe
                  yarndf = pd.concat([yarndf, pd.DataFrame([{'Project name': pattern_data['pattern']['name'],
                                                            'Yarn id': yarnData['id'],
                                                            'Crochet gauge': yarnData['crochet_gauge'],
                                                            'Knit gauge': yarnData['knit_gauge'],
                                                            'Yarn name': yarnData['name'],
                                                            'ply': yarnData['ply'], 'wpi': yarnData['wpi'],
                                                            'Min yardage': int(yardage[0]),
                                                            'Max yardage': int(yardage[1])}])], ignore_index=True)

                  # Get key information from pattern details for recommender
                  keydf = pd.concat([keydf, pd.DataFrame([{'Project name': pattern_data['pattern']['name'],
                                                          'Project id': pattern_data['pattern']['id'],
                                                          'Difficulty average': pattern_data['pattern']['difficulty_average'],
                                                          'UK': pattern_data['pattern']['has_uk_terminology'],
                                                          'US' : pattern_data['pattern']['has_us_terminology']}])], ignore_index=True)
                except:
                  print("No yarn weight recorded")
      else:
        print("No patterns found")
  else:
    print("Unable to access patterns")

  keydf = keydf.merge(yarndf, on='Project name')
  print(keydf)
  return keydf


def fetchInventory():
  # Open stored inventory into dataframe
  with open("inventory.json", "r") as file:
    inventory = json.load(file)
    inv = pd.DataFrame(inventory)
    groupedInv = inv.groupby(['Yarn name','ply', 'wpi', 'Total yardage']).agg(
        min_yardage=('Total yardage', 'min'),
        max_yardage=('Total yardage', 'max')).reset_index()

    print(groupedInv)
    return groupedInv

def filterPatterns(groupedInv, patterns):
  # Filters patterns for yarn available in inventory
  filtereddf = patterns[patterns['Yarn name'].isin(groupedInv['Yarn name'])]
  finaldf = pd.DataFrame(columns = patterns.columns)
  for idx, row in filtereddf.iterrows():
    rowYardage = row['Max yardage']
    maxYardage = int(max(groupedInv.loc[groupedInv['Yarn name'] == row['Yarn name'], 'max_yardage'].values))

    if (rowYardage > 0) & (rowYardage <= maxYardage):
      # Directly adds if empty for future compatibility
      if finaldf.empty:
        finaldf = pd.DataFrame([row])
      else:
        finaldf = pd.concat([finaldf, pd.DataFrame([row])], ignore_index = True)

  print(finaldf)

patterns = getPatterns()


No yarn weight recorded
No yarn weight recorded
No yarn weight recorded
No yarn weight recorded
                       Project name  Project id  Difficulty average     UK  \
0            Color Block Market Bag      783725            2.152466   None   
1          Cozy Days Daisy Cardigan     7298622            3.785714   None   
2                   Sungold Blanket     7305100            2.000000   True   
3         Gingerbread Christmas CAL     7392532            5.357143   True   
4                  Chestnut Sweater     7407210            0.000000  False   
5               Molly's Maine Mitts     7412204            0.000000   None   
6                    Easy Soap Cozy      878811            1.909091   None   
7                   Mulberry Afghan     1238236            2.591398   None   
8                 Mini Pop Pullover     1333224            2.538462   None   
9                  Mistborn Coaster     7394843            2.100000   None   
10          Virus Shawl | Virustuch      56579

In [77]:
# Text pre-processing before recommender
chars = r'[^\w\s]' # Code for removing special characters
patterns['Project name'] = patterns['Project name'].replace(chars, '', regex=True)

# Get current yarn inventory and filter through patterns
groupedInv = fetchInventory()
filterPatterns(groupedInv, patterns)

   Yarn name  ply  wpi  Total yardage  min_yardage  max_yardage
0         DK    8   11            500          500          500
1         DK    8   12            200          200          200
2  Fingering    4   14            800          800          800
       Project name  Project id  Difficulty average    UK    US  Yarn id  \
0    Easy Soap Cozy      878811            1.909091  None  True       11   
1  Mistborn Coaster     7394843            2.100000  None  True       11   
2       Petal Party     7412962            0.000000  None  True       11   

  Crochet gauge Knit gauge Yarn name ply wpi  Min yardage  Max yardage  
0          None         22        DK   8  11           50           55  
1          None         22        DK   8  11           30           40  
2          None         22        DK   8  11           77          109  


In [81]:
### Basic recommender ###
# Take the last 5 patterns saved by the user and use the average calculated metrics to then recommend
# In release, should use the saved and completed patterns
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity


def readSaved():
  with open("saved.json", "r") as file:
    savedPatterns = json.load(file)
    saved = pd.DataFrame(savedPatterns)
    return saved

def vectorize(names, bow, vectorizer):
  tfidfTransformer = TfidfTransformer()
  tfidfMatrix = tfidfTransformer.fit_transform(bow)
  columnNames = vectorizer.get_feature_names_out() # Gets words for column naming

  return pd.DataFrame(tfidfMatrix.toarray())

def transformPatterns(saved, patterns):
  savedNames = saved['Project name'].tolist()
  patternNames = patterns['Project name'].tolist()
  features = ['Project name', 'Difficulty average', 'US', 'UK', "Yarn id"]

  savedCombined = saved.copy()
  patternsCombined = patterns.copy()

  def combineFeatures(data):
    return data['Project name'] + ' ' + str(data['Difficulty average']) + ' ' + str(data['US']) + ' ' + str(data['UK']) + ' ' + str(data['Yarn id'])

  for feature in features:
    savedCombined[feature] = savedCombined[feature].fillna('')
    patternsCombined[feature] = patternsCombined[feature].fillna('')

  savedCombined['combinedFeatures'] = savedCombined.apply(combineFeatures, axis=1)
  patternsCombined['combinedFeatures'] = patternsCombined.apply(combineFeatures, axis=1)
  # Combines all words so features from saved and patterns are used in the vectorising process
  allCombined = pd.concat([savedCombined["combinedFeatures"], patternsCombined["combinedFeatures"]])

  vectorizer = CountVectorizer()
  bagOfWordsMatrix = vectorizer.fit_transform(allCombined)

  # Splits the combined vectorizing to compare
  savedMatrix = vectorize(savedNames, bagOfWordsMatrix[:len(saved)], vectorizer)
  patternMatrix = vectorize(patternNames, bagOfWordsMatrix[len(saved):], vectorizer)
  #bagOfWordsMatrix = vectorizer.fit_transform(savedCombined["combinedFeatures"])
  #one = vectorize(savedNames,bagOfWordsMatrix, vectorizer)
  #bagOfWordsMatrix = vectorizer.transform(patternsCombined["combinedFeatures"]) # Maps to the saved pattern words
  #two = vectorize(patternNames, bagOfWordsMatrix, vectorizer)

  # Finds similaritys
  similar = cosine_similarity(patternMatrix, savedMatrix)
  similarDf = pd.DataFrame(similar, index = patternNames, columns = savedNames)
  print(similarDf)


saved = readSaved()
transformPatterns(saved, patterns)


                                 Dante the Dragon  Cute Critter Dragon  \
Color Block Market Bag                   0.033947             0.132593   
Cozy Days Daisy Cardigan                 0.035716             0.024733   
Sungold Blanket                          0.248950             0.172400   
Gingerbread Christmas CAL                0.036589             0.025338   
Chestnut Sweater                         0.263681             0.182601   
Mollys Maine Mitts                       0.039405             0.027288   
Easy Soap Cozy                           0.036589             0.025338   
Mulberry Afghan                          0.040900             0.159752   
Mini Pop Pullover                        0.037430             0.025921   
Mistborn Coaster                         0.050695             0.035107   
Virus Shawl  Virustuch                   0.034859             0.024140   
Jellyfish Babies                         0.042750             0.029605   
Positive Potato                       

Notes for retrieving names for the recommender
*   Remove any symbols from pattern names cause having " causes issues with the json
*   Any missing data should be 0 or none
* Probably include links to the projects when saving to the json


